In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import os

# Path to dataset
baseFolder = "C:/Users/User/OneDrive/Documents/SignLanguageApp/SLangDataset/data"
image_size = 224  # MobileNetV2 input size
batch_size = 4
num_classes = len(os.listdir(baseFolder))  # Number of classes (letters)

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,  # Use 20% of data for validation
)

# Load training data
train_generator = train_datagen.flow_from_directory(
    baseFolder,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode="categorical",
    subset="training",
)

# Load validation data
val_generator = train_datagen.flow_from_directory(
    baseFolder,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation",
)

# Load the MobileNetV2 model
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(image_size, image_size, 3))

# # Freeze the base model
# base_model.trainable = False

# # Add custom layers for classification
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dropout(0.5)(x)  # Regularization
# x = Dense(128, activation="relu")(x)
# x = Dropout(0.5)(x)
# predictions = Dense(num_classes, activation="softmax")(x)

# # Create the final model
# model = Model(inputs=base_model.input, outputs=predictions)

# # Compile the model
# model.compile(optimizer=Adam(learning_rate=0.001), loss="categorical_crossentropy", metrics=["accuracy"])

# # Train the model
# epochs = 30
# history = model.fit(
#     train_generator,
#     epochs=epochs,
#     validation_data=val_generator,
# )

# # Save the model
# model.save("sign_language_mobilenetv2.h5")
model = tf.keras.models.load_model("sign_language_mobilenetv2.h5")
# Fine-tune the model (unfreeze some base model layers)
base_model.trainable = True
fine_tune_at = 100  # Unfreeze from this layer onwards

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Recompile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=1e-5), loss="categorical_crossentropy", metrics=["accuracy"])

# Fine-tuning
fine_tune_epochs = 10
total_epochs = epochs + fine_tune_epochs
history_fine = model.fit(
    train_generator,
    epochs=total_epochs,
    initial_epoch=history.epoch[1],
    validation_data=val_generator,
)

# Save the fine-tuned model
model.save("sign_language_mobilenetv2_fine_tuned.h5")



Found 43200 images belonging to 27 classes.
Found 10800 images belonging to 27 classes.
Epoch 2/40
 1498/10800 [===>..........................] - ETA: 3:28 - loss: 0.1744 - accuracy: 0.9549

KeyboardInterrupt: 

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import os

# Path to dataset
baseFolder = "C:/Users/User/OneDrive/Documents/SignLanguageApp/SLangDataset/data"
image_size = 224  # Input size for ResNet
batch_size = 32
num_classes = len(os.listdir(baseFolder))  # Number of classes (letters)

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,  # Use 20% of data for validation
)

# Load training data
train_generator = train_datagen.flow_from_directory(
    baseFolder,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode="categorical",
    subset="training",
)

# Load validation data
val_generator = train_datagen.flow_from_directory(
    baseFolder,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation",
)

# Load the ResNet50 model
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(image_size, image_size, 3))

# Freeze the base model
base_model.trainable = False

# Add custom layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Regularization
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation="softmax")(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
epochs = 20
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator,
)

# Save the model
model.save("sign_language_resnet50.h5")

# Fine-tune the model (unfreeze some base model layers)
base_model.trainable = True
fine_tune_at = 100  # Unfreeze from this layer onwards

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Recompile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=1e-5), loss="categorical_crossentropy", metrics=["accuracy"])

# Fine-tuning
fine_tune_epochs = 10
total_epochs = epochs + fine_tune_epochs
history_fine = model.fit(
    train_generator,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1],
    validation_data=val_generator,
)

# Save the fine-tuned model
model.save("sign_language_resnet50_fine_tuned.h5")

Found 43200 images belonging to 27 classes.
Found 10800 images belonging to 27 classes.
Epoch 1/20
1350/1350 [==============================] - 300s 221ms/step - loss: 2.6574 - accuracy: 0.1597 - val_loss: 1.9378 - val_accuracy: 0.3586
Epoch 2/20
1350/1350 [==============================] - 292s 217ms/step - loss: 2.1267 - accuracy: 0.2696 - val_loss: 1.6999 - val_accuracy: 0.4494
Epoch 3/20
1350/1350 [==============================] - 293s 217ms/step - loss: 1.9834 - accuracy: 0.3105 - val_loss: 1.5697 - val_accuracy: 0.4853
Epoch 4/20
1350/1350 [==============================] - 289s 214ms/step - loss: 1.8914 - accuracy: 0.3386 - val_loss: 1.4648 - val_accuracy: 0.5179
Epoch 5/20
1350/1350 [==============================] - 290s 215ms/step - loss: 1.8479 - accuracy: 0.3561 - val_loss: 1.3960 - val_accuracy: 0.5506
Epoch 6/20
1350/1350 [==============================] - 296s 219ms/step - loss: 1.8057 - accuracy: 0.3639 - val_loss: 1.3508 - val_accuracy: 0.5418
Epoch 7/20
1350/1350 [==

In [2]:
import cv2
from cvzone.HandTrackingModule import HandDetector
from cvzone.ClassificationModule import Classifier
import numpy as np
import math
cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=1)
# classifier = Classifier("Model/keras_model.h5", "Model/labels.txt")
classifier = Classifier("C:/Users/User/OneDrive/Documents/SignLanguageApp/ModelCode/sign_language_mobilenetv2.h5")
offset = 20
imgSize = 300
counter = 0
labels = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "Space", "T", "U", "V", "W", "X", "Y", "Z"]
while True:
    success, img = cap.read()
    if success:
        imgOutput = img.copy()
        hands, img = detector.findHands(img)
        if hands:
            hand = hands[0]
            x, y, w, h = hand['bbox']
            imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255

            # Ensure the cropping boundaries are within the image dimensions
            y1, y2 = max(0, y - offset), min(img.shape[0], y + h + offset)
            x1, x2 = max(0, x - offset), min(img.shape[1], x + w + offset)
            imgCrop = img[y1:y2, x1:x2]

            imgCropShape = imgCrop.shape
            if imgCropShape[0] > 0 and imgCropShape[1] > 0:  # Ensure the cropped image is not empty
                aspectRatio = h / w
                if aspectRatio > 1:
                    k = imgSize / h
                    wCal = math.ceil(k * w)
                    imgResize = cv2.resize(imgCrop, (wCal, imgSize))
                    imgResizeShape = imgResize.shape
                    wGap = math.ceil((imgSize - wCal) / 2)
                    imgWhite[:, wGap:wCal + wGap] = imgResize
                    prediction, index = classifier.getPrediction(imgWhite, draw=False)
                    print(prediction, index)
                else:
                    k = imgSize / w
                    hCal = math.ceil(k * h)
                    imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                    imgResizeShape = imgResize.shape
                    hGap = math.ceil((imgSize - hCal) / 2)
                    imgWhite[hGap:hCal + hGap, :] = imgResize
                    prediction, index = classifier.getPrediction(imgWhite, draw=False)
            if 0 <= index < len(labels):
                cv2.rectangle(imgOutput, (x - offset, y - offset-50),
                              (x - offset+90, y - offset-50+50), (255, 0, 255), cv2.FILLED)
                cv2.putText(imgOutput, labels[index], (x, y -26), cv2.FONT_HERSHEY_COMPLEX, 1.7, (255, 255, 255), 2)
                cv2.rectangle(imgOutput, (x-offset, y-offset),
                              (x + w+offset, y + h+offset), (255, 0, 255), 4)
            cv2.imshow("ImageCrop", imgCrop)
            cv2.imshow("ImageWhite", imgWhite)
        cv2.imshow("Image", imgOutput)
    cv2.waitKey(1)
    cv2.imshow("Image", imgOutput)
    cv2.waitKey(1)

No Labels Found


NameError: name 'imgOutput' is not defined